In [8]:
import cv2
import mediapipe as mp

# Inicializar MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen del sombrero y convertirla a RGBA
hat = cv2.imread('hat-crop.png', cv2.IMREAD_UNCHANGED)
hat_h, hat_w, _ = hat.shape  # Obtener dimensiones del sombrero

# Cargar la imagen del collar y convertirla a RGBA
ruff = cv2.imread('ruff-crop.png', cv2.IMREAD_UNCHANGED)
ruff_h, ruff_w, _ = hat.shape  # Obtener dimensiones del collar

# Variables de estado anteriores para suavizado
prev_x, prev_y, prev_w, prev_h = None, None, None, None
alpha = 0.2  # Factor de suavizado; entre más cercano a 1, menos suavizado

# Inicializar la cámara o cargar un video
cap = cv2.VideoCapture(0)  # Usa '0' para la cámara web

with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("No se puede acceder a la cámara.")
            break

        # Convertir la imagen a RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesar la imagen para detectar caras
        results = face_detection.process(image_rgb)

        # Convertir de nuevo a BGR para OpenCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        # Dibujar los resultados de la detección en la imagen
        if results.detections:
            for detection in results.detections:
                # Dibuja el cuadro de la cara detectada
                #mp_drawing.draw_detection(image_bgr, detection)

                # Extraer el bounding box de la detección
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image_bgr.shape  # Dimensiones de la imagen

                # Convertir coordenadas relativas a píxeles absolutos
                x = int(bboxC.xmin * iw)
                y = int(bboxC.ymin * ih)
                w = int(bboxC.width * iw)
                h = int(bboxC.height * ih)

                # Suavizado exponencial
                if prev_x is not None:
                    x = int(alpha * x + (1 - alpha) * prev_x)
                    y = int(alpha * y + (1 - alpha) * prev_y)
                    w = int(alpha * w + (1 - alpha) * prev_w)
                    h = int(alpha * h + (1 - alpha) * prev_h)
                
                # Actualizar las variables anteriores
                prev_x, prev_y, prev_w, prev_h = x, y, w, h

               # Redimensionar el sombrero y calcular posición centrada
                new_hat_w = int(w * 1.3)  # Sombrero más ancho que la cara
                new_hat_h = int(hat_h * (new_hat_w / hat_w))  # Mantener proporción
                resized_hat = cv2.resize(hat, (new_hat_w, new_hat_h), interpolation=cv2.INTER_AREA)
                
                # Calcular la posición del sombrero centrado
                x_hat = x - int((new_hat_w - w) / 2)  # Centrar el sombrero horizontalmente
                y_hat = y - new_hat_h  # Posicionar el sombrero justo encima de la cara

                # Ajustar dimensiones para evitar que el sombrero se salga de los límites de la imagen
                hat_h_end = min(ih, y_hat + new_hat_h) - y_hat
                hat_w_end = min(iw, x_hat + new_hat_w) - x_hat
                resized_hat_cropped = resized_hat[:hat_h_end, :hat_w_end]

                # Superponer el sombrero con transparencia
                if y_hat >= 0:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] = \
                            image_bgr[y_hat:y_hat + hat_h_end, x_hat:x_hat + hat_w_end, i] * (1 - resized_hat_cropped[:, :, 3] / 255.0) + \
                            resized_hat_cropped[:, :, i] * (resized_hat_cropped[:, :, 3] / 255.0)
                

                #Redimensionar el collar y calcular posición centrada
                new_ruff_w = int(w * 1.5)  # Collar más ancho que la cara
                new_ruff_h = int(ruff_h * (new_ruff_w / ruff_w))  # Mantener proporción
                resized_ruff = cv2.resize(ruff, (new_ruff_w, new_ruff_h), interpolation=cv2.INTER_AREA)

                # Calcular la posición del collar centrado
                x_ruff = x - int((new_ruff_w - w) / 2)  # Centrar el collar horizontalmente
                y_ruff = y + h - int(0.2 * new_ruff_h)  # Posicionar el collar justo debajo de la cara

                # Ajustar dimensiones para evitar que el collar se salga de los límites de la imagen
                ruff_h_end = min(ih, y_ruff + new_ruff_h) - y_ruff
                ruff_w_end = min(iw, x_ruff + new_ruff_w) - x_ruff
                resized_ruff_cropped = resized_ruff[:ruff_h_end, :ruff_w_end]

                # Superponer el collar con transparencia
                if y_ruff + ruff_h_end <= ih:
                    for i in range(0, 3):  # BGR canales
                        image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] = \
                            image_bgr[y_ruff:y_ruff + ruff_h_end, x_ruff:x_ruff + ruff_w_end, i] * (1 - resized_ruff_cropped[:, :, 3] / 255.0) + \
                            resized_ruff_cropped[:, :, i] * (resized_ruff_cropped[:, :, 3] / 255.0)
        
        # Mostrar la imagen en una ventana
        cv2.imshow('Face Detection', image_bgr)

        # Presionar 'q' para salir
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar las ventanas
cap.release()
cv2.destroyAllWindows()
